In [1]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import time
import re

In [2]:
def seoul_crawling(start_date, end_date):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    url_1 = "https://search.seoul.co.kr/index.php?scope=&sort=&cpCode=seoul;nownewsiCategory=&pCategory=undefined"
    kor_url_head = url_1 + "&period=&sDate="+str(start_date)+"&eDate="+str(end_date)+"&keyword=카이스트&pageNum="
    eng_url_head = url_1+ "&period=&sDate="+str(start_date)+"&eDate="+str(end_date)+"&keyword=kaist&pageNum="

    # 검색 결과에 따른 url 링크 저장
    url1 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    page = 1
    while True:
        url_final = eng_url_head + str(page)
        try:
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.no')) != 0:
                break
            elements = soup.select('.article a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            url1.append(link)
        page += 1
            
    # 검색 결과에 따른 url2 링크 저장
    url2 = []
    page = 1
    while True:
        url_final = kor_url_head + str(page)
        try:
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.no')) != 0:
                break
            elements = soup.select('.article a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            url2.append(link)
        page += 1
        
    url = list(set(url1).union(set(url2)))
    if len(url)==0:
        print("Articles Not Found")
        return 0
    
    df = pd.DataFrame({"url": url}, columns=["url"])
    print("Total number of articles: ", len(df))
    
    contents = []
    titles = []
    urls = []
    for row in tqdm(df.itertuples()):
        link = getattr(row, 'url')
        response = requests.get(link, headers=headers)
        html = response.content.decode('euc-kr','replace')
        soup = bs(html,'html.parser')
        try:
            for script in soup(["script", "style"]):
                script.decompose() 
            title = soup.select('title')[0].get_text()
            if 'go.seoul' not in link:
                content = soup.select('.S20_v_article')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            else:
                content = soup.select('.atic_txt1')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
            title = title.replace('\xa0', " ")
            title = title.replace('\n', " ")
            titles.append(title)
            contents.append(content)
            urls.append(link)
        except:
            print(row)
    df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
    df.to_excel("articles/seoul.xlsx",index = False)
    df.to_pickle("pickle files/seoul.pkl")

In [3]:
seoul_crawling('2015.01.01', '2020.10.23')

0it [00:00, ?it/s]

Total number of articles:  3432


6it [00:02,  2.77it/s]

Pandas(Index=5, url='http://www.seoul.co.kr/news/newsView.php?id=20161024500059')


7it [00:02,  2.31it/s]

...

3361it [20:14,  3.24it/s]

Pandas(Index=3360, url='http://www.seoul.co.kr/news/newsView.php?id=20190904500157')


3419it [20:34,  3.07it/s]

Pandas(Index=3418, url='http://www.seoul.co.kr/news/newsView.php?id=20190111500026')


3432it [20:41,  2.76it/s]
